In [ ]:
import pandas as pd
import spacy
import re
import os

# Carga el modelo de spaCy que descargaste
nlp = spacy.load("en_core_web_sm")

# Define la ruta del archivo de datos
project_dir = os.path.join(os.path.dirname(__file__), '..', '..')
file_path = os.path.join(project_dir, 'data', 'raw', 'pubmed_articles.csv')
print(f"Cargando datos desde: {file_path}")

# Carga los datos extraídos en un DataFrame de pandas
df = pd.read_csv(file_path)

# --- Preprocesamiento de texto ---
def preprocess_text(text):
    """
    Realiza la limpieza y normalización del texto.
    """
    # 1. Limpieza inicial: elimina caracteres especiales y URLs
    # Usa expresiones regulares para limpiar texto
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S*@\S*\s?', '', text) # Elimina emails
    text = re.sub(r'[^\w\s]', '', text) # Elimina puntuación

    # 2. Tokenización y lematización
    # Procesa el texto con spaCy
    doc = nlp(text.lower()) 
    
    # 3. Eliminación de palabras vacías y lematización
    # Excluye palabras vacías (stopwords) y símbolos, y usa la forma base de la palabra (lemma_)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    
    # 4. Reconstrucción del texto limpio
    return " ".join(tokens)

# Aplica la función de preprocesamiento a la columna 'abstract'
# y guarda el resultado en una nueva columna 'clean_abstract'
df['clean_abstract'] = df['abstract'].apply(preprocess_text)

# Muestra los resultados para comparar
print("\n--- Abstract Original ---")
print(df['abstract'][0])
print("\n--- Abstract Preprocesado ---")
print(df['clean_abstract'][0])

# Opcional: Guarda el DataFrame con la nueva columna para el siguiente paso
clean_data_path = os.path.join(project_dir, 'data', 'processed', 'clean_pubmed_articles.csv')
os.makedirs(os.path.dirname(clean_data_path), exist_ok=True)
df.to_csv(clean_data_path, index=False)
print(f"\nDatos preprocesados guardados en: {clean_data_path}")